In [4]:
import nibabel as nib
import numpy as np
import fnmatch
import pandas as pd
import os

In [5]:
def calculate_bounding_box(segmentation_path):
    segmentation_mask = nib.load(segmentation_path).get_fdata()

    # Assume the tumor is labeled with '1'
    tumor_mask = segmentation_mask == 4

    # Find the indices where the tumor is present
    tumor_indices = np.argwhere(tumor_mask)

    if tumor_indices.size == 0:
        return {
            'file': segmentation_path,
            'length': 0,
            'width': 0,
            'height': 0
        }

    # Get the minimum and maximum indices along each axis
    min_coords = tumor_indices.min(axis=0)
    max_coords = tumor_indices.max(axis=0)

    # Calculate the size along each dimension
    tumor_length = max_coords[0] - min_coords[0]  # x-axis (Length)
    tumor_width = max_coords[1] - min_coords[1]   # y-axis (Width)
    tumor_height = max_coords[2] - min_coords[2]  # z-axis (Height)

    # Get the voxel dimensions from the header (in mm)
    voxel_dimensions = nib.load(segmentation_path).header.get_zooms()

    # Convert the voxel counts to physical dimensions in mm
    tumor_length_mm = tumor_length * voxel_dimensions[0]
    tumor_width_mm = tumor_width * voxel_dimensions[1]
    tumor_height_mm = tumor_height * voxel_dimensions[2]

    return {
        'file': segmentation_path,
        'length': tumor_length_mm,
        'width': tumor_width_mm,
        'height': tumor_height_mm
    }

def list_all_files(root_dir, prefix):
    matching_folders = []
    for root, dirs, files in os.walk(root_dir):
        for dir_name in dirs:
            if fnmatch.fnmatch(dir_name, f'{prefix}*'):
                matching_folders.append(f'dataset/BraTS2021_Training_Data/{dir_name}/{dir_name}_seg.nii.gz')
    return matching_folders

In [6]:
root_directory = 'dataset/BraTS2021_Training_Data'
prefix = 'BraTS2021_'
folders = list_all_files(root_directory, prefix)

result_df = pd.DataFrame({ 'file': [], 'length': [], 'width': [], 'height': []})

for folder in folders:
    segmentation_mask = nib.load(folder).get_fdata()
    data = calculate_bounding_box(folder)

    result_df.loc[len(result_df)] = data

result_df.to_csv('4-tumor-bounding-box.csv')